<a href="https://colab.research.google.com/github/Untick/InspectrumClinic_RecSys/blob/main/Paul%20Himyak/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import uuid
import re
import numpy as np
import seaborn as sns
from datetime import datetime

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import gdown

In [2]:
dataset_url = "https://drive.google.com/uc?export=download&id=1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg"
#dataset_url = "https://docs.google.com/spreadsheets/d/1ErXvN290PPthLGsb0ALpa_0aUDnvC0jg/edit?usp=share_link&ouid=113369956649761615778&rtpof=true&sd=true"
dataset_filename = "profpatolog_modified.xlsx"
#!wget -qc dataset_url -O dataset_filename
gdown.download(dataset_url, output=dataset_filename, quiet=True )


'profpatolog_modified.xlsx'

In [3]:
df = pd.read_excel("profpatolog_modified.xlsx")


In [4]:
#Удаляем случайные вбросы
df.drop([843, 851, 1022, 1024], inplace = True)

#Убираем пустые колонки
empty_columns = df.columns[df.isna().all()]
df.drop(empty_columns, inplace = True, axis = 1)

In [5]:
FIELD_CLIENT_AGE = "КлиентВозраст"
FIELD_CLIENT_BIRTH_DAY = "КлиентДатаРождения"
FIELD_CLIENT_GENDER = "КлиентПол"
FIELD_CLIENT_GENDER_M = 0
FIELD_CLIENT_GENDER_F = 1
FIELD_CONCLUSION = "ЗаключениеМК"
FIELD_CONCLUSION_FIT = 0
FIELD_CONCLUSION_PART_FIT = 1
FIELD_CONCLUSION_NOT_FIT = 2

FIELD_DISPANSARY_GROUP = "ДиспансернаяГруппа"
FIELD_DISPANSARY_GROUP_I = 0
FIELD_DISPANSARY_GROUP_II = 1
FIELD_DISPANSARY_GROUP_III = 2
FIELD_DISPANSARY_GROUP_OTHER = 3

FIELD_HARMFUL_FACTORS = "ВредныеФакторы"

In [6]:
mkb_columns = df.filter(regex="(?i)МКБ10").columns


In [28]:
mkb_columns

Index(['ПсихиатрияНаркология1_МКБ101', 'Офтальмология1_МКБ101',
       'Оториноларингология1_МКБ101', 'ХирургНеИсп1_МКБ101',
       'Гинекология1_МКБ101', 'Стоматология1_МКБ101',
       'Дерматовенерология1_МКБ101', 'Рентгенология1_МКБ101',
       'Неврология1_МКБ101', 'Терапия1_МКБ101', 'Хирургия1_МКБ101',
       'Офтальмология1_МКБ102', 'Терапия2_МКБ101', 'Неврология2_МКБ101',
       'ЛабораторныеИсследования1_МКБ101', 'ЭКГНеИсп1_МКБ101',
       'УЗИДиагностика1_МКБ101', 'УЗИДиагностика2_МКБ101',
       'ПсихиатрияНаркология2_МКБ101', 'Терапия1_МКБ102',
       'ЛабораторныеИсследования2_МКБ101', 'Терапия1_МКБ103',
       'Психиатрия1_МКБ101', 'Гинекология1_МКБ102', 'Рентгенология2_МКБ101',
       'Гинекология2_МКБ101', 'Хирургия2_МКБ101',
       'ПроцедурнаяМедсестра1_МКБ101', 'ФункциональнаяДиагностика1_МКБ101',
       'УЗИДиагностика1_МКБ102', 'Стоматология2_МКБ101', 'Неврология1_МКБ102',
       'Оториноларингология2_МКБ101', 'Дерматовенерология1_МКБ102',
       'Дерматовенерология

In [27]:
df[mkb_columns]

,ПсихиатрияНаркология1_МКБ101,Офтальмология1_МКБ101,Оториноларингология1_МКБ101,ХирургНеИсп1_МКБ101,Гинекология1_МКБ101,Стоматология1_МКБ101,Дерматовенерология1_МКБ101,Рентгенология1_МКБ101,Неврология1_МКБ101,Терапия1_МКБ101,...,Психиатрия1_МКБ102,Психиатрия2_МКБ101,Психиатрия2_МКБ102,Рентгенология1_МКБ102,Рентгенология2_МКБ102,Психиатрия3_МКБ101,Психиатрия3_МКБ102,Психиатрия4_МКБ101,Психиатрия4_МКБ102,Оториноларингология3_МКБ101
0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,Z00.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Z00.0,Z00.0,NaN,NaN,NaN,NaN,Z00.0,Z00.0,Z00.0,I11.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Z00.0,H52.1,NaN,NaN,NaN,NaN,Z00.0,NaN,NaN,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,Z00.0,Z00.0,Z00.0,NaN,Z00.0,NaN,NaN,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,H52.2,Z00.0,NaN,NaN,Z00.0,NaN,NaN,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1979,Z00.0,Z00.0,Z00.0,NaN,NaN,K02.1,Z00.0,NaN,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1980,Z00.0,NaN,Z00.0,NaN,Z00.0,NaN,Z00.0,NaN,Z00.0,Z00.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:

# объединяем значения из нескольких столбцов в один массив, удаляем дубликаты
df['D'] = df.apply(lambda row: list(set([row[mkb_col_name] for mkb_col_name in mkb_columns if not pd.isna(row[mkb_col_name]) ])), axis=1).apply(lambda x: x if len(x) > 0 else None)


In [37]:
df['D']

0              [Z00.0]
1       [Z00.0, I11.9]
2       [Z00.0, H52.1]
3              [Z00.0]
4       [H52.2, Z00.0]
             ...      
1977              None
1978              None
1979    [K02.1, Z00.0]
1980    [Z00.0, I11.9]
1981           [Z00.0]
Name: D, Length: 1978, dtype: object

In [34]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: x.replace(".", "x"))

In [29]:
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].str.split(",")
#df[FIELD_HARMFUL_FACTORS] = df[FIELD_HARMFUL_FACTORS].apply(lambda x: ",".join(x))

In [7]:
tokenizer = Tokenizer(filters=" ", split = ",", oov_token = "unknown", char_level = False)

In [8]:
tokenizer.fit_on_texts(df[FIELD_HARMFUL_FACTORS])

In [9]:
xTrainHarmfulIndexes = tokenizer.texts_to_sequences(df[FIELD_HARMFUL_FACTORS])

In [26]:
max_len = max(xTrainHarmfulIndexes, key=len)


13


In [17]:
xTrainHarmfulIndexesPadded = pad_sequences(xTrainHarmfulIndexes, maxlen=max_len, padding='post')

In [14]:
xTrainHarmfulIndexesPadded.shape

(1978, 20)